In [1]:
from Angrybird import AngryBird
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
env=AngryBird()

# 240 coeffs

In [2]:
coeffs_temp = np.load("css.npy")

In [3]:
coeffs = np.ones((10, 4, 6))
for i in range(10):
    for j in range(4):
        coeffs[i][j] = coeffs_temp[i]

In [4]:
seconds = np.arange(11)
pos = np.zeros((11, 2))
pos[:, 0] = seconds * 10
pos[:, 1] = seconds * 20 - 2 * seconds**2

In [5]:
mean_states = np.load("mean_states.npy")

In [6]:
sqr2 = 1/np.sqrt(2)
rot = np.array([[sqr2, -sqr2],
               [sqr2, sqr2]])

In [7]:
quantiles = np.load("quantiles.npy")
print(quantiles[:, 5])
print(np.searchsorted(quantiles[:, 5],2, side="left"))

[-1.18906167 -0.00613677  1.18385758]
3


In [10]:
def main(state, coeffs):
    t = int(state[0])
    a, _ = rot.dot(state[1:] - mean_states[t])
    i = np.searchsorted(quantiles[:, t],a, side="left")
    return coeffs[t, i, :2]*pos[t]-coeffs[t, i, 2:4]*state[1:] + coeffs[t, i, 4:]

In [ ]:
from time import time
nul=np.zeros(2)
m = 100
n = 1000
vents = np.load("vents.npy")
ecart = 0.001
cos_min = 400.
while cos_min > 100:
    t = -time()
    for i in range(10):
        for l in range(4):
            for k in range(6):
                coeff_change = np.linspace(coeffs[i][l][k]-ecart, coeffs[i][l][k]+ecart, 9)
                costs = np.zeros(9)
                for j in range(9):
                    coeffs[i][l][k] = coeff_change[j]
                    for z in range(m):
                        state = env.reset()
                        env.vent = vents[z]
                        for _ in range(10):
                            force = main(state, coeffs)
                            state, _, _, _ = env.step(force)
                        env.step(nul)
                        costs[j] += env.cost.sum()
                coeffs[i][l][k] = coeff_change[np.argmin(costs)]
    cos = 0.
    for z in range(n):
        state = env.reset(False)
        env.vent = vents[z]
        for _ in range(10):
            force = main(state, coeffs)
            state, _, _, _ = env.step(force)
        env.step(nul)
        cos += env.cost.sum()
    cos /= n
    t += time()
    print(m, n, ecart, costs.min()/m, cos, t/60)
    if cos < cos_min:
        cos_min = cos
        np.save("l", coeffs)
    else:
        cos_min = cos + 20
        m += 100
        n += 100
        ecart /= 2

100 1000 0.001 110.76998933132208 116.99631678940135 8.900758477052053
100 1000 0.001 110.47087912263167 116.80092310161888 8.912276136875153
100 1000 0.001 110.3814111340934 116.7111927592063 8.959009210268656
100 1000 0.001 110.33047887029007 116.67054236882616 8.93382913271586
100 1000 0.001 110.29727732984486 116.64922437241125 8.912242261568705


In [ ]:
vents = np.load("vents.npy")
mean_states = np.load("mean_states.npy")
nul=np.zeros(2)
costs = 0.
m = 1100
for z in tqdm(range(m)):
    state = env.reset(False)
    env.vent = vents[z]
    for y in range(10):
        force = main(state, coeffs)
        state, _, _, _ = env.step(force)
    plt.scatter(state[1], state[2])
    state, _, _, _ = env.step(nul)
    costs += env.cost.sum()
print(costs/m)